In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
entries = []
results = []
constructDataset(entries, results, seasonsData, qualiResultsData, driversData, constructorsData, enginesData)


In [4]:
#print(entries)

In [5]:
#print(results)

In [6]:
X = np.array(entries)
y = results
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.5631738175701411

In [7]:
reg.coef_

array([ 0.60651233,  0.65189826,  0.24933791, -0.04814837,  0.03937488,
        0.01792253])

In [8]:
#Prediction

(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [9]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#print(newDrivers)
driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('../F1PredictWeb/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [10]:
circuit = 32
circuitName = "Mexican"

with open('../F1PredictWeb/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [11]:
driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('../F1PredictWeb/predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 55.2 %
	2: 43.4 %
	3: 0.4 %
	4: 0.1 %
	10: 0.3 %
	15: 0.2 %
	20: 0.4 %
Fernando Alonso (McLaren):
	16: 0.2 %
	17: 7.2 %
	18: 29.7 %
	19: 56.2 %
	20: 6.7 %
Kimi Räikkönen (Ferrari):
	2: 1.7 %
	3: 45.7 %
	4: 52.0 %
	10: 0.2 %
	15: 0.3 %
	20: 0.1 %
Sebastian Vettel (Ferrari):
	2: 2.0 %
	3: 53.0 %
	4: 44.4 %
	10: 0.1 %
	15: 0.3 %
	20: 0.2 %
Romain Grosjean (Haas F1 Team):
	4: 0.1 %
	5: 0.5 %
	6: 6.4 %
	7: 17.6 %
	8: 22.3 %
	9: 16.6 %
	10: 11.9 %
	11: 8.9 %
	12: 7.0 %
	13: 5.0 %
	14: 2.6 %
	15: 0.7 %
	16: 0.1 %
	20: 0.3 %
Nico Hülkenberg (Renault):
	6: 0.3 %
	7: 3.1 %
	8: 8.3 %
	9: 13.6 %
	10: 17.5 %
	11: 19.9 %
	12: 15.4 %
	13: 12.1 %
	14: 6.1 %
	15: 3.1 %
	16: 0.5 %
	20: 0.1 %
Sergio Pérez (Force India):
	4: 0.2 %
	5: 7.0 %
	6: 26.7 %
	7: 32.5 %
	8: 13.0 %
	9: 8.7 %
	10: 3.7 %
	11: 4.1 %
	12: 2.0 %
	13: 1.5 %
	14: 0.3 %
	15: 0.2 %
	20: 0.1 %
Daniel Ricciardo (Red Bull):
	5: 0.9 %
	6: 5.4 %
	7: 16.5 %
	8: 23.1 %
	9: 16.9 %
	10: 12.4 %
	11: 10.1 %
	12: 7.2 %
	

In [12]:
predictedEntrants = []

for did, cid in newDrivers.items():
    if circuit not in drivers[did].trackpwr:
        drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in drivers[did].constructor.trackpwr:
        drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in drivers[did].constructor.engine.trackpwr:
        drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        drivers[did].pwr,
        drivers[did].constructor.pwr, 
        drivers[did].constructor.engine.pwr,
        drivers[did].trackpwr[circuit],
        drivers[did].constructor.trackpwr[circuit],
        drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)
    

In [13]:
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[-1.26850673  0.62420558 -0.97034938 -0.90999958 -0.11381795  0.11479028
 -0.07951752 -0.07264757 -1.20496461  0.03038687  0.44827995 -0.16426045
  0.08521493  0.79872522 -0.11475939  0.64147323  0.18856407  0.31669258
  0.56726919  0.13439738]


In [14]:
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(drivers[int(did)].name, drivers[int(did)].constructor.name, predictedResults[index]))

Lewis Hamilton (Mercedes): -1.2685067344906285
Fernando Alonso (McLaren): 0.6242055825462443
Kimi Räikkönen (Ferrari): -0.9703493762647797
Sebastian Vettel (Ferrari): -0.9099995818083368
Romain Grosjean (Haas F1 Team): -0.1138179513959085
Nico Hülkenberg (Renault): 0.11479028430321425
Sergio Pérez (Force India): -0.07951752241164217
Daniel Ricciardo (Red Bull): -0.0726475724408147
Valtteri Bottas (Mercedes): -1.2049646069194642
Kevin Magnussen (Haas F1 Team): 0.030386873914927864
Marcus Ericsson (Sauber): 0.44827994759841727
Max Verstappen (Red Bull): -0.16426044802870485
Carlos Sainz (Renault): 0.08521492949079339
Stoffel Vandoorne (McLaren): 0.7987252215840888
Esteban Ocon (Force India): -0.11475938833110882
Lance Stroll (Williams): 0.6414732279309701
Pierre Gasly (Toro Rosso): 0.18856406863576808
Brendon Hartley (Toro Rosso): 0.31669258370628095
Sergey Sirotkin (Williams): 0.567269190427691
Charles Leclerc (Sauber): 0.1343973837071708
